In [5]:
"""
Generate a tidy dataset of mean energy expenditure and equivalent nectar volume per bee,
per behaviour, and per flower species.

Assumptions:
- Each bee has a unique BeeID
- Each bee has one body mass value (in g) recorded as 'weight_g'
- Energy expenditure per bout is precomputed and stored in 'Energy_J'

Required columns in input:
- BeeID
- event_type (e.g. 'buzz' or 'takeoff'; also accepts 'flight' as takeoff)
- weight_g
- Energy_J
"""

import pandas as pd

# --- Load bout-level data and standardise ---
df = pd.read_csv("C:/Users/labadmin/Documents/Uppsala analyses/bee_bout_power_output.csv")
df["event_type"] = df["event_type"].replace("flight", "takeoff")
df["body_mass_mg"] = df["weight_g"] * 1000
df["BeeID_clean"] = df["BeeID"].str.strip().str.lower()

# --- Load colony info and clean ---
bee_info = pd.read_excel("C:/Users/labadmin/Documents/Uppsala analyses/beeID laser file name + weight.xlsx")
bee_info["BeeID_clean"] = bee_info["BeeID"].astype(str).str.strip().str.lower()
bee_info_subset = bee_info[["BeeID_clean", "ColonyID"]]

# --- Step 1: Aggregate energy by BeeID and behaviour ---
bee_means = df.groupby(["BeeID_clean", "event_type"]).agg({
    "Energy_J": "mean",
    "Power_W": "mean",
    "body_mass_mg": "first"
}).reset_index()

# --- Step 2: Merge ColonyID ---
bee_means = bee_means.merge(bee_info_subset, on="BeeID_clean", how="left")

# --- Step 3: Define nectar energy content (J/µl) per flower ---
nectar_energy = {
    "sunflower": 9.5819,
    "borage": 12.4025,
    "raspberry": 8.3663
}

# --- Step 4: Compute nectar volume for each flower ---
records = []
for flower, energy_per_ul in nectar_energy.items():
    for _, row in bee_means.iterrows():
        records.append({
            "bee_id": row["BeeID_clean"],
            "colony_id": row["ColonyID"],
            "body_mass_mg": row["body_mass_mg"],
            "event_type": row["event_type"],
            "mean_energy_J": row["Energy_J"],
            "flower_type": flower,
            "nectar_uL_per_event": row["Energy_J"] / energy_per_ul,
            "nectar_uL_per_s": row["Power_W"]  / energy_per_ul
        })

# --- Step 5: Create tidy DataFrame and export ---
tidy_df = pd.DataFrame(records)

# Save to CSV
tidy_df.to_csv("C:/Users/labadmin/Documents/Uppsala analyses/nectar_volume_per_event_and_per_second.csv", index=False)

# Preview result
tidy_df.head()

,bee_id,colony_id,body_mass_mg,event_type,mean_energy_J,flower_type,nectar_uL_per_event,nectar_uL_per_s
0,green3,3,181.0,buzz,0.069996,sunflower,0.007305,0.003841
1,green3,3,181.0,takeoff,0.087094,sunflower,0.009089,0.006711
2,green4,4,216.5,buzz,0.097090,sunflower,0.010133,0.008142
3,green4,4,216.5,takeoff,0.109043,sunflower,0.011380,0.009105
4,green5,5,266.5,buzz,0.098550,sunflower,0.010285,0.006207
